# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [ ]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [ ]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29
import math
def gradf(a, b, c):
    df_da = -3 * a**2 - 0.5 * a**(-0.5)
    df_db = 3 * math.cos(3 * b) + 2.5 * b**1.5
    df_dc = 1 / c**2
    return [df_da, df_db, df_dc]

# expected answer is the list of

yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")



OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [ ]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# To estimate the gradient numerically, we can use a method called finite
# difference approximation. This method estimates the derivative of a function
# at a point by calculating the change in the function's output over a small
# change in the input.

import math

def f(a, b, c):
    return -a**3 + math.sin(3*b) - 1.0/c + b**2.5 - a**0.5

def numerical_gradient(f, a, b, c, h=1e-5):
    numerical_grad = [0, 0, 0]


# Estimate df/da
    a_plus_h = f(a + h, b, c)
    a_minus_h = f(a - h, b, c)
    numerical_grad[0] = (a_plus_h - a_minus_h) / (2 * h)

    # Estimate df/db
    b_plus_h = f(a, b + h, c)
    b_minus_h = f(a, b - h, c)
    numerical_grad[1] = (b_plus_h - b_minus_h) / (2 * h)

    # Estimate df/dc
    c_plus_h = f(a, b, c + h)
    c_minus_h = f(a, b, c - h)
    numerical_grad[2] = (c_plus_h - c_minus_h) / (2 * h)

    return numerical_grad

    # Calculate the numerical gradient
numerical_grad = numerical_gradient(f, 2, 3, 4)

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390820336
OK for dim 1: expected 10.25699027111255, yours returns 10.256990271617639
OK for dim 2: expected 0.0625, yours returns 0.06250000001983835


In [ ]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

def symmetric_numerical_gradient(f, a, b, c, h=1e-5):
    numerical_grad2 = [0, 0, 0]

    # Symmetric derivative for df/da
    a_plus_h = f(a + h, b, c)
    a_minus_h = f(a - h, b, c)
    numerical_grad2[0] = (a_plus_h - a_minus_h) / (2 * h)

    # Symmetric derivative for df/db
    b_plus_h = f(a, b + h, c)
    b_minus_h = f(a, b - h, c)
    numerical_grad2[1] = (b_plus_h - b_minus_h) / (2 * h)

    # Symmetric derivative for df/dc
    c_plus_h = f(a, b, c + h)
    c_minus_h = f(a, b, c - h)
    numerical_grad2[2] = (c_plus_h - c_minus_h) / (2 * h)

    return numerical_grad2

    # Calculate the symmetric numerical gradient
numerical_grad2 = symmetric_numerical_gradient(f, 2, 3, 4)

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390820336
OK for dim 1: expected 10.25699027111255, yours returns 10.256990271617639
OK for dim 2: expected 0.0625, yours returns 0.06250000001983835


## section 2: support for softmax

In [ ]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out


  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward

    return out

  def __neg__(self):
    out = Value(-self.data, (self,), '-')

    def _backward():
      self.grad += -1 * out.grad
    out._backward = _backward

    return out

  def __sub__(self, other):
    return self + (-other)

  def __truediv__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    return self * other**-1

  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
      self.grad += (other * self.data**(other - 1)) * out.grad
    out._backward = _backward

    return out

  def exp(self):
    out = Value(exp(self.data), (self,), 'exp')

    def _backward():
      self.grad += out.data * out.grad
    out._backward = _backward

    return out

  def log(self):
    out = Value(log(self.data), (self,), 'log')

    def _backward():
      self.grad += (1 / self.data) * out.grad
    out._backward = _backward

    return out

  def backward(self): # exactly as in video
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()




In [ ]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
    # Calculate the exponentials of the logits (keeps them as Value objects)
    counts = [logit.exp() for logit in logits]

    # Sum the exponentials (as a Value object)
    denominator = sum(counts, Value(0.0))

    # Calculate the softmax probabilities (keeps them as Value objects)
    out = [count / denominator for count in counts]

    return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log()  # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

# Softmax and negative log likelihood loss function
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log()  # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


2.1755153626167147
2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [ ]:
import torch

# Initialize the logits as torch tensors with requires_grad=True
logits = torch.tensor([0.0, 3.0, -2.0, 1.0], requires_grad=True)

# Compute the softmax (this creates a non-leaf tensor)
softmax_probs = torch.softmax(logits, dim=0)

# Compute the negative log likelihood loss
loss = -torch.log(softmax_probs[3])

# Perform backpropagation
loss.backward()

# Print the loss value
print(f"Loss: {loss.item()}")

# Print the gradients of the leaf tensors
for i in range(len(logits)):
    print(f"Gradient for logit[{i}]: {logits.grad[i].item()}")


Loss: 2.1755154132843018
Gradient for logit[0]: 0.041772566735744476
Gradient for logit[1]: 0.8390244841575623
Gradient for logit[2]: 0.005653302650898695
Gradient for logit[3]: -0.8864504098892212
